In [2]:
!pip install imageio

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import imageio
import os
import shutil

# --- PARÁMETROS DEL ROBOT (Tus medidas) ---
L1 = 8.5  # Longitud Hombro
L2 = 7.5  # Longitud Codo
H_BASE = 5.0 # Altura ficticia de la base para visualización 3D
Z_MAX_FISICO = 5.0 # Altura física máxima del actuador Z

# --- FUNCIONES AUXILIARES PARA 3D ---

def get_rotation_matrix_z(theta_deg):
    """Calcula matriz de rotación básica alrededor del eje Z"""
    theta = np.radians(theta_deg)
    c = np.cos(theta)
    s = np.sin(theta)
    return np.array([[c, -s, 0],
                     [s,  c, 0],
                     [0,  0, 1]])

def calcular_cadena_cinematica(q1, q2, d3_servo):
    """
    Calcula las posiciones (x,y,z) y orientaciones (matrices R)
    de TODAS las articulaciones para dibujarlas en 3D.
    """
    # 1. Convertir d3 servo (0-180) a distancia física Z (cm)
    # Si d3=0 -> Z baja 0cm. Si d3=180 -> Z baja 5cm.
    z_desplazamiento = d3_servo * (Z_MAX_FISICO / 180.0)
    
    # --- Puntos Clave (Joints) ---
    # P0: Base en el suelo
    p0 = np.array([0, 0, 0])
    # P1: Hombro (altura de la base)
    p1 = np.array([0, 0, H_BASE])
    
    # Matriz de rotación del Hombro (q1)
    R1 = get_rotation_matrix_z(q1)
    # P2: Codo
    p2 = p1 + R1 @ np.array([L1, 0, 0])
    
    # Matriz de rotación global del Codo (q1 + q2)
    R2 = get_rotation_matrix_z(q1 + q2)
    # P3: Efector final (Parte superior, antes de bajar)
    p3 = p2 + R2 @ np.array([L2, 0, 0])
    
    # P4: Punta del efector final (después del actuador prismático)
    # El actuador se mueve en el eje Z negativo del sistema local,
    # pero como es SCARA RRP simple, siempre es Z global hacia abajo.
    p4 = p3 - np.array([0, 0, z_desplazamiento])

    # Lista de puntos y sus matrices de orientación asociadas
    joints = [p0, p1, p2, p3, p4]
    # Orientaciones: Base(I), Hombro(R1), Codo(R2), Punta(R2)
    orientations = [np.eye(3), R1, R2, R2] 
    
    return joints, orientations

def plot_frame_arrows(ax, origin, R, length=2.0):
    """Dibuja las flechas RGB (XYZ) en un punto dado"""
    # Ejes X, Y, Z locales rotados
    x_vec = R @ np.array([length, 0, 0])
    y_vec = R @ np.array([0, length, 0])
    z_vec = R @ np.array([0, 0, length])
    
    # Dibujar flechas (Quivers)
    # X = Rojo, Y = Verde, Z = Azul
    ax.quiver(origin[0], origin[1], origin[2], x_vec[0], x_vec[1], x_vec[2], color='r', linewidth=1.5, arrow_length_ratio=0.2)
    ax.quiver(origin[0], origin[1], origin[2], y_vec[0], y_vec[1], y_vec[2], color='g', linewidth=1.5, arrow_length_ratio=0.2)
    ax.quiver(origin[0], origin[1], origin[2], z_vec[0], z_vec[1], z_vec[2], color='b', linewidth=1.5, arrow_length_ratio=0.2)

# --- FUNCIÓN PRINCIPAL DE GENERACIÓN DE GIF 3D ---

def generar_gif_scara_3d_con_pausas(lista_poses_clave, nombre_archivo='scara_3d_pausas.gif'):
    print(f"🎬 Iniciando simulación 3D avanzada: {nombre_archivo}...")
    
    # Configuración de directorios temporales
    temp_dir = 'temp_frames_3d'
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir) # Limpiar si existe
    os.makedirs(temp_dir)

    filenames = []
    frame_count = 0
    
    # --- PARÁMETROS DE ANIMACIÓN ---
    pasos_interpolacion = 20  # Cuadros entre puntos (suavidad)
    cuadros_pausa = 15        # Cuadros para "congelar" la imagen (pausa)

    # --- GENERACIÓN DE TRAYECTORIA COMPLETA ---
    trayectoria_full = []
    
    for i in range(len(lista_poses_clave)):
        start_pose = lista_poses_clave[i]
        
        # 1. Agregar la pose actual y sus repeticiones (PAUSA)
        for _ in range(cuadros_pausa):
            trayectoria_full.append(start_pose)
            
        # 2. Si hay una siguiente pose, interpolar hacia ella
        if i < len(lista_poses_clave) - 1:
            end_pose = lista_poses_clave[i+1]
            
            # Interpolación lineal para cada articulación
            q1_interp = np.linspace(start_pose[0], end_pose[0], pasos_interpolacion)
            q2_interp = np.linspace(start_pose[1], end_pose[1], pasos_interpolacion)
            d3_interp = np.linspace(start_pose[2], end_pose[2], pasos_interpolacion)
            
            for j in range(pasos_interpolacion):
                trayectoria_full.append((q1_interp[j], q2_interp[j], d3_interp[j]))

    total_frames = len(trayectoria_full)
    print(f"📊 Total de cuadros a generar: {total_frames}")

    # --- BUCLE DE RENDERIZADO 3D ---
    for i, (q1, q2, d3) in enumerate(trayectoria_full):
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        
        # Calcular física del robot
        joints, Rs = calcular_cadena_cinematica(q1, q2, d3)
        p0, p1, p2, p3, p4 = joints
        
        # --- DIBUJAR ESLABONES (Visualización tipo esqueleto) ---
        # Base a Hombro (Poste)
        ax.plot([p0[0], p1[0]], [p0[1], p1[1]], [p0[2], p1[2]], 'k-', linewidth=4, alpha=0.6)
        # Hombro a Codo (L1) - Color Rojo suave
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], '-', color='#e74c3c', linewidth=6, alpha=0.8)
        # Codo a Efector Top (L2) - Color Azul suave
        ax.plot([p2[0], p3[0]], [p2[1], p3[1]], [p2[2], p3[2]], '-', color='#3498db', linewidth=6, alpha=0.8)
        # Efector Top a Punta (Actuador Prismático) - Color Verde
        ax.plot([p3[0], p4[0]], [p3[1], p4[1]], [p3[2], p4[2]], '-', color='#2ecc71', linewidth=4)

        # --- DIBUJAR MARCOS DE COORDENADAS (Flechas RGB) ---
        plot_frame_arrows(ax, p1, Rs[1], length=3.0) # Frame Hombro
        plot_frame_arrows(ax, p2, Rs[2], length=3.0) # Frame Codo
        plot_frame_arrows(ax, p4, Rs[3], length=2.0) # Frame Punta

        # --- CONFIGURACIÓN DEL ESPACIO 3D ---
        limit = L1 + L2 + 2
        ax.set_xlim(-limit, limit)
        ax.set_ylim(-limit, limit)
        ax.set_zlim(0, H_BASE + 2)
        ax.set_xlabel('X (cm)')
        ax.set_ylabel('Y (cm)')
        ax.set_zlabel('Z (cm)')
        title_text = f'Simulación SCARA RRP 3D\nFrame {i}/{total_frames}\n'
        title_text += f'Q1={q1:.0f}°, Q2={q2:.0f}°, D3 (Servo)={d3:.0f}'
        ax.set_title(title_text)
        
        # Vista isométrica para que se vea bien
        ax.view_init(elev=30, azim=45)

        # Guardar frame
        filename = os.path.join(temp_dir, f'frame_{frame_count:03d}.png')
        plt.savefig(filename, dpi=80) # DPI bajo para velocidad
        filenames.append(filename)
        frame_count += 1
        plt.close(fig)
        
        if i % 10 == 0 or i == total_frames - 1:
             print(f"\rRenderizando: {i+1}/{total_frames} cuadros...", end="")

    print("\nCompilando GIF (esto puede tomar un momento)...")
    # Crear GIF (usando v2 para evitar warnings)
    with imageio.get_writer(nombre_archivo, mode='I', duration=0.08) as writer:
        for filename in filenames:
            image = imageio.v2.imread(filename)
            writer.append_data(image)
            
    # Limpieza
    shutil.rmtree(temp_dir)
    print(f"✅ GIF 3D con pausas guardado: {nombre_archivo}")

# ==========================================
# DEFINICIÓN DE LOS 3 PUNTOS CLAVE (POSES)
# ==========================================
# Formato: (q1_grados, q2_grados, d3_servo_0-180)

poses_demo = [
    (0, 90, 0),    # Pose 1: Brazo doblado a la derecha, Arriba
    (90, 0, 180),  # Pose 2: Brazo extendido al frente, Abajo (Pick)
    (-45, -45, 90) # Pose 3: Brazo retraído a la izquierda, Altura media
]

# ¡EJECUTAR LA MAGIA!
generar_gif_scara_3d_con_pausas(poses_demo, 'scara_3d_avanzado.gif')

🎬 Iniciando simulación 3D avanzada: scara_3d_avanzado.gif...
📊 Total de cuadros a generar: 85
Renderizando: 85/85 cuadros...
Compilando GIF (esto puede tomar un momento)...
✅ GIF 3D con pausas guardado: scara_3d_avanzado.gif
